In [1]:
#Importação dos pacotes usados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
from scipy import signal as sg
import winsound
import time
from IPython.display import display
from ipywidgets import widgets

#Distância DTW

#conda install rpy2 #instalador do rpy2
#rpy2.robjects.r('install.packages("dtw")') #instalador do pacote dtw do R
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.functions import SignatureTranslatedFunction
rpy2.robjects.numpy2ri.activate()
R = rpy2.robjects.r
DTW = importr('dtw')

####AVB_register: Função de cadastro de nova voz

def AVB_register(audio, ID):
    T_frame = 1.5 #tempo em segundos de quebra
    threshold = 0.13 #Valor de identificação de nulos
    fv = [0.00625*2,0.425*2] # faixa de frequência da voz normalizado (dividido por TA)
    contador = 0
    
    #Preciso definir o tamanho do reshape
    n_colunas = int(TA*T_frame)
    zeros_add = len(audio)%n_colunas
    zeros_add = int(n_colunas - zeros_add)
    zeros_add = np.zeros(zeros_add)
    audio = np.concatenate((audio,zeros_add), axis = None)
    n_linhas = int(len(audio)/n_colunas)
    frame = audio.reshape((n_linhas, n_colunas))
    
    for a in range(n_linhas):
        audio_frame = frame[a,]
        
        #Filtro Butterworth
        
        f1, f2 = sg.butter(8, fv, btype = 'bandpass')
        audio_BW = sg.lfilter(f1, f2, audio_frame)
        
        #Filtro Offset
        
        audio_OFF = audio_BW - np.mean(audio_BW)
        
        #Normalização e removação de threshold
        
        audio_NM = audio_OFF/np.amax(audio_OFF)
        audio_NM[np.absolute(audio_NM) < threshold] = 0
        
        #RLE
        
        rpy2.robjects.r(''' rle_1 <- function(audio) {
            rle_audio <- rle(audio)
            rle_audio$values[ rle_audio$values==0 & rle_audio$lengths > 100] <- NA
            audio <- inverse.rle(rle_audio)
            audio <- audio[!is.na(audio)]
            return(audio)
            } ''')
        funcao_rle = rpy2.robjects.globalenv['rle_1']       
        audio_RLE = rpy2.robjects.FloatVector(audio_NM)
        audio_RLE = np.array(funcao_rle(audio_RLE))
        print("Tamanho_audio:", len(audio_RLE))
        
         
        #MFCC
        
        if len(audio_RLE) > (TA/2):
            from python_speech_features import mfcc #não sei o porquê, mas funciona só se coloca aqui
            mfcc = mfcc(audio_RLE,TA)
            (n_linhas, n_colunas) = mfcc.shape
            mfcc = mfcc.reshape((1,n_linhas*n_colunas))
            
            #Adicionando o ID e apendando no MFCC geral
            
            id_mfcc = np.append(ID,mfcc)
            AVB_CDB.append(id_mfcc)
            
            contador = 1
    if contador == 1:
        print("Usuário cadastrado com sucesso")
    else:
        print("Usuário não cadastrado. Fale novamente")
            
####AVB Finder - Encontra voz numa lista cadastrada

def AVB_finder(audio, ID, AVB_CDB, precisao):
    t = time.time()
    T_frame = 1.5 #tempo em segundos de quebra
    threshold = 0.13 #Valor de identificação de nulos
    fv = [0.00625*2,0.425*2] # faixa de frequência da voz normalizado (dividido por TA)
    resposta = []
    AVB_CDB_ID = list()
    n_linhas = len(AVB_CDB)
    for a in range(n_linhas):
        if AVB_CDB[a][0] == ID:
            AVB_CDB_ID.append(AVB_CDB[a][1:])
    n_linhas_ID = len(AVB_CDB_ID)
    if n_linhas_ID == 0:
        return("Usuário não cadastrado")           
    #Preciso definir o tamanho do reshape
    n_colunas = int(TA*T_frame)
    zeros_add = len(audio)%n_colunas
    zeros_add = int(n_colunas - zeros_add)
    zeros_add = np.zeros(zeros_add)
    audio = np.concatenate((audio,zeros_add), axis = None)
    n_linhas = int(len(audio)/n_colunas)
    frame = audio.reshape((n_linhas, n_colunas))
    
    for a in range(n_linhas):
        audio_frame = frame[a,]
        
        #Filtro Butterworth
        
        f1, f2 = sg.butter(8, fv, btype = 'bandpass')
        audio_BW = sg.lfilter(f1, f2, audio_frame)
        
        #Filtro Offset
        
        audio_OFF = audio_BW - np.mean(audio_BW)
        
        #Normalização e removação de threshold
        
        max_audio = np.amax(audio_OFF)
        print("Max_audio: ",max_audio)
        if max_audio == 0.0:
            continue
        
        audio_NM = audio_OFF/max_audio #Aqui tá dando warning 148
        audio_NM[np.absolute(audio_NM) < threshold] = 0
        
        #RLE
        
        rpy2.robjects.r(''' rle_1 <- function(audio) {
            rle_audio <- rle(audio)
            rle_audio$values[ rle_audio$values==0 & rle_audio$lengths > 100] <- NA
            audio <- inverse.rle(rle_audio)
            audio <- audio[!is.na(audio)]
            return(audio)
            } ''')
        funcao_rle = rpy2.robjects.globalenv['rle_1']
        audio_RLE = rpy2.robjects.FloatVector(audio_NM)
        audio_RLE = np.array(funcao_rle(audio_RLE))
        print("Tamanho_audio:", len(audio_RLE))
        
        #MFCC
        
        if len(audio_RLE) >= (TA*0.4):
            from python_speech_features import mfcc #não sei o porquê, mas funciona só se coloca aqui
            mfcc = mfcc(audio_RLE,TA)
            (n_linhas, n_colunas) = mfcc.shape
            mfcc = mfcc.reshape((1,n_linhas*n_colunas))
            
            for b in range(n_linhas_ID):
                mfcc_ID = AVB_CDB_ID[b].astype('float64')
                res_mfcc1 = rpy2.robjects.FloatVector(mfcc_ID)
                res_mfcc2 = rpy2.robjects.FloatVector(mfcc[0])
                rpy2.robjects.r('''
                    dtw_dist <- function(MFCC1, MFCC2) {
                    return(dtw(MFCC1,MFCC2)$normalizedDistance)
                    }
                    ''')
                r_f = rpy2.robjects.globalenv['dtw_dist']
                dist = r_f(res_mfcc1,res_mfcc2)
                print("Distancia:", dist[0])
                if dist[0] < precisao:
                     resposta = np.concatenate((resposta,1), axis = None)
                else:
                    resposta = np.concatenate((resposta,0), axis = None)
        else:
            resposta = np.concatenate((resposta,0), axis = None)
            print("Sem voz suficiente")
    if np.mean(resposta) > 0.5:
        return("Usuário identificado")
    else:
        return("Usuário não identificado", np.mean(resposta))
    #print("Tempo de execução: ",time.time() - t)






In [2]:
import os
from scipy.io import wavfile

#Importa arquivos gravados pelo whatsapp

AVB_CDB = list() #limpa lista
T_gravacao = 4 #tempo em segundos da gravação
TA = 8000 #frequência de amostragem
endereco = "C:/Users/edube/Documents/Archangel/whatsapp_wav_8k/"
arquivos = os.listdir(endereco)

for a in range(len(arquivos)):
    nome = arquivos[a]
    fs, audio = wavfile.read(endereco + nome)
    AVB_register(audio, nome)

Tamanho_audio: 9020
Tamanho_audio: 5233
Tamanho_audio: 8023
Tamanho_audio: 5341
Tamanho_audio: 8736
Tamanho_audio: 8144
Usuário cadastrado com sucesso
Tamanho_audio: 3326
Tamanho_audio: 9149
Tamanho_audio: 9346
Tamanho_audio: 8049
Tamanho_audio: 8400
Tamanho_audio: 4641
Tamanho_audio: 9693
Tamanho_audio: 8859
Tamanho_audio: 6912
Tamanho_audio: 1208
Usuário cadastrado com sucesso
Tamanho_audio: 11159
Tamanho_audio: 3927
Tamanho_audio: 2654
Tamanho_audio: 11099
Tamanho_audio: 3609
Tamanho_audio: 3544
Tamanho_audio: 4229
Tamanho_audio: 7303
Tamanho_audio: 10228
Usuário cadastrado com sucesso
Tamanho_audio: 379
Tamanho_audio: 9890
Tamanho_audio: 9055
Tamanho_audio: 6173
Tamanho_audio: 8749
Tamanho_audio: 11089
Tamanho_audio: 7050
Tamanho_audio: 5035
Tamanho_audio: 6849
Tamanho_audio: 7486
Tamanho_audio: 6130
Tamanho_audio: 6506
Usuário cadastrado com sucesso
Tamanho_audio: 9016
Tamanho_audio: 6645
Tamanho_audio: 8839
Tamanho_audio: 5592
Tamanho_audio: 2745
Tamanho_audio: 6535
Tamanho_audio

Tamanho_audio: 7088
Tamanho_audio: 9885
Tamanho_audio: 5884
Tamanho_audio: 3413
Tamanho_audio: 8695
Tamanho_audio: 8993
Tamanho_audio: 3193
Usuário cadastrado com sucesso
Tamanho_audio: 5314
Tamanho_audio: 7662
Tamanho_audio: 6867
Tamanho_audio: 6186
Tamanho_audio: 6334
Tamanho_audio: 9303
Tamanho_audio: 2752
Tamanho_audio: 1284
Usuário cadastrado com sucesso
Tamanho_audio: 6884
Tamanho_audio: 5797
Tamanho_audio: 7647
Tamanho_audio: 7645
Tamanho_audio: 8179
Tamanho_audio: 9092
Tamanho_audio: 3210
Tamanho_audio: 6643
Tamanho_audio: 2705
Tamanho_audio: 12000
Tamanho_audio: 6288
Tamanho_audio: 2059
Usuário cadastrado com sucesso
Tamanho_audio: 8223
Tamanho_audio: 7639
Tamanho_audio: 6979
Tamanho_audio: 879
Usuário cadastrado com sucesso
Tamanho_audio: 12000
Tamanho_audio: 7583
Tamanho_audio: 2629
Tamanho_audio: 7527
Tamanho_audio: 7584
Tamanho_audio: 9344
Tamanho_audio: 2878
Tamanho_audio: 8155
Tamanho_audio: 7198
Tamanho_audio: 8312
Tamanho_audio: 4454
Tamanho_audio: 7371
Tamanho_audio: 

Tamanho_audio: 3643
Tamanho_audio: 4426
Tamanho_audio: 8051
Tamanho_audio: 1382
Usuário cadastrado com sucesso
Tamanho_audio: 11237
Tamanho_audio: 3412
Tamanho_audio: 7737
Tamanho_audio: 2195
Tamanho_audio: 5600
Tamanho_audio: 7819
Tamanho_audio: 9812
Tamanho_audio: 8899
Tamanho_audio: 5442
Usuário cadastrado com sucesso
Tamanho_audio: 7068
Tamanho_audio: 7524
Tamanho_audio: 3073
Tamanho_audio: 630
Usuário cadastrado com sucesso
Tamanho_audio: 6110
Tamanho_audio: 5494
Tamanho_audio: 10310
Tamanho_audio: 5331
Tamanho_audio: 8769
Tamanho_audio: 9322
Tamanho_audio: 6025
Tamanho_audio: 7865
Tamanho_audio: 3796
Tamanho_audio: 2566
Tamanho_audio: 7346
Usuário cadastrado com sucesso
Tamanho_audio: 3153
Tamanho_audio: 10566
Tamanho_audio: 4157
Tamanho_audio: 6579
Tamanho_audio: 6380
Tamanho_audio: 6000
Tamanho_audio: 11090
Tamanho_audio: 1754
Usuário cadastrado com sucesso
Tamanho_audio: 8266
Tamanho_audio: 6381
Tamanho_audio: 8058
Tamanho_audio: 7141
Tamanho_audio: 8374
Tamanho_audio: 9651
Ta

In [4]:
AVB_CDB = list() #limpa lista
T_gravacao = 4 #tempo em segundos da gravação
TA = 8000 #frequência de amostragem

from tkinter import *

#GUI e voz gravada

class Application:
    def __init__(self, master=None):
        
        self.fontePadrao = ("Arial", "12") #define uma fonte padrão
        
        self.C1 = Frame(master)
        self.C1["pady"] = 20
        self.C1.pack()
        
          
        self.C2 = Frame(master)
        self.C2["padx"] = 20
        self.C2.pack()
  
        self.C3 = Frame(master)
        self.C3["pady"] = 10
        self.C3.pack()
  
        self.C4 = Frame(master)
        self.C4["pady"] = 10
        self.C4["padx"] = 0
        self.C4.pack()
        
        self.C5 = Frame(master)
        self.C5["pady"] = 10
        self.C5["padx"] = 0
        self.C5.pack()
                
        #Título  
        self.titulo = Label(self.C1, text="!!!Bem vindo ao sistema Archangel!!!")
        self.titulo["font"] = ("Arial", "15", "bold")
        self.titulo.pack()
        
        #Nome
  
        self.menu = Label(self.C2,text="Nome", font=self.fontePadrao)
        self.menu.pack(side = LEFT)

        #Botões
  
        self.register = Button(self.C3)
        self.register["text"] = "Cadastrar nova voz"
        self.register["width"] = 17
        self.register["font"] = self.fontePadrao
        self.register["command"] = self.AVB_register
        self.register.pack(side=LEFT)
        self.finder = Button(self.C3)
        self.finder["text"] = "Encontrar minha voz"
        self.finder["font"] = self.fontePadrao
        self.finder["width"] = 17
        self.finder["command"] = self.AVB_finder
        self.finder.pack(side = LEFT)
          
       #Caixa de texto
        
        self.txtbox = Entry(self.C2)
        self.txtbox["width"] = 30
        self.txtbox["font"] = self.fontePadrao
        self.txtbox.pack(side = LEFT)      
    
        #Respostas
        
        self.beep = Label(self.C4,text="Ao som do beep diga seu nome", font=self.fontePadrao)
        self.beep.pack(side = LEFT)  
        self.answer = Label(self.C5,text="", font=self.fontePadrao)
        self.answer.pack()
    
    #Método verificar senha
    def AVB_register(self):
        usuario = self.txtbox.get()
        winsound.Beep(1800, 700)
        som = sd.rec(int(T_gravacao * TA), samplerate=TA, channels=1)
        sd.wait()
        AVB_register(som, usuario)
        self.answer["text"] = "Voz cadastrada"
    def AVB_finder(self):
        usuario = self.txtbox.get()
        winsound.Beep(1800, 700)
        som1 = sd.rec(int(T_gravacao * TA), samplerate=TA, channels=1)
        sd.wait()
        resposta = AVB_finder(som1, usuario, AVB_CDB, 6)
        self.answer["text"] = resposta
 
root = Tk()
Application(root)
root.title("archangel")
root.mainloop()

Tamanho_audio: 8964
Tamanho_audio: 6636
Tamanho_audio: 8013
Usuário cadastrado com sucesso
Max_audio:  0.2656401795422896
Tamanho_audio: 8392
Distancia: 3.4357840957877355
Distancia: 3.7363470222219872
Distancia: 4.070539087201064
Max_audio:  0.06078826095364535
Tamanho_audio: 6966
Distancia: 4.003468399612475
Distancia: 3.5539783566557377
Distancia: 4.292188423442477
Max_audio:  0.003427752889829591
Tamanho_audio: 8015
Distancia: 4.066188349112094
Distancia: 4.0274074364884855
Distancia: 2.7353124097032766
Max_audio:  0.31216403610197396
Tamanho_audio: 6126
Distancia: 4.044289870243007
Distancia: 4.002361156591887
Distancia: 4.268695630348057
Max_audio:  0.09486585003192706
Tamanho_audio: 8123
Distancia: 4.189857519773455
Distancia: 4.009067333790147
Distancia: 4.634767961015993
Max_audio:  0.001703970398929206
Tamanho_audio: 8077
Distancia: 4.098438880163962
Distancia: 3.9209703763129955
Distancia: 2.8964541372933494


In [ ]:
nome = arquivos[1]
fs, audio = wavfile.read(endereco + nome)
#nome = arquivos[2]
AVB_finder(audio, nome, AVB_CDB, 2.5)